# Decoding

In [1]:
# Need to install ImageAnalysis3 and run the appropriate Startup_py3.py
%run "C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\required_files\Startup_py3.py"
# append path to import ImageAnalysis3
sys.path.append(r"C:\Users\cosmosyw\Documents\Softwares")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

38620


## 0.1 Folders

In [2]:
save_folder = r'E:\DNA_analysis\20220819_Mecp2_DNA_from_0809'
pre_process_folder = r'Z:\Cosmos_DNA\MeC2_0809_DNA_preprocessed'

save_filenames = [os.path.join(pre_process_folder, _fl) for _fl in os.listdir(pre_process_folder)
                  if _fl.split(os.extsep)[-1]=='hdf5']
# extract fov_id
save_fov_ids = [int(os.path.basename(_fl).split('.hdf5')[0].split('_')[-1]) for _fl in save_filenames]

debug = False

print(f"{len(save_filenames)} fovs detected")

analysis_folder = os.path.join(save_folder, 'Analysis_0508')

segmentation_folder = os.path.join(analysis_folder, 'Segmentation')
if not os.path.exists(segmentation_folder):
    os.makedirs(segmentation_folder)
    print(f"Creating segmentation_folder: {segmentation_folder}")
else:
    print(f"Use segmentation_folder: {segmentation_folder}")

cand_spot_folder = os.path.join(analysis_folder, 'CandSpots')
if not os.path.exists(cand_spot_folder):
    os.makedirs(cand_spot_folder)
    print(f"Creating cand_spot_folder: {cand_spot_folder}")
else:
    print(f"Use cand_spot_folder: {cand_spot_folder}")

decoder_folder = cand_spot_folder.replace('CandSpots', 'Decoder')
if debug:
    _version = 0
    while os.path.exists(os.path.join(decoder_folder, f'v{_version}')):
        _version += 1
    decoder_folder = os.path.join(decoder_folder, f'v{_version}')
if not os.path.exists(decoder_folder):
    os.makedirs(decoder_folder)
    print(f"Creating decoder_folder: {decoder_folder}")
else:
    print(f"Use decoder_folder: {decoder_folder}")
    
picked_folder = os.path.join(analysis_folder, 'Picked_v0')
if not os.path.exists(picked_folder):
    os.makedirs(picked_folder)
    print(f"Creating picked_folder: {picked_folder}")
else:
    print(f"Use picked_folder: {picked_folder}")

243 fovs detected
Use segmentation_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation
Use cand_spot_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\CandSpots
Use decoder_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder
Use picked_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Picked_v0


# 1. Decoding of DNA-MERFISH

In [3]:
# load two codebooks
import pandas as pd

ctp11_codebook = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-1000_codebook_probe.csv', header=0)
ctp13_codebook = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\Summary_tables\CTP13-mop_SE_1000.csv', header=0)

In [4]:
ctp11_codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3742742-3759944,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.0
1,1:6245958-6258969,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.0
2,1:8740008-8759916,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2.0
3,1:11247744-11257616,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3.0
4,1:13741888-13757922,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,X:161247067-161259813,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52.0
984,X:163750534-163758706,1058,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,X,53.0
985,X:166247682-166259932,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54.0
986,X:168746045-168757590,1060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,55.0


In [5]:
ctp13_codebook

,name,id,NDB_376,NDB_578,NDB_400,NDB_533,NDB_568,NDB_464,NDB_466,NDB_431,...,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604,chr,chr_order
0,1:9627926-9637875,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.0
1,1:9799472-9811359,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1.0
2,1:20920666-20929358,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,2.0
3,1:21522568-21534512,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3.0
4,1:21732182-21745770,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,X:101961397-101974103,987,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,2.0
962,X:136808741-136818926,988,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,3.0
963,X:140517829-140527051,989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,4.0
964,X:167157164-167167452,990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,5.0


## Load bit_2_name

In [6]:
readout_df_gn = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-mouse-genome-1000_combo_readouts.csv')
readout_df_se = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\Summary_tables\CTP13-mouse-SE_combo_readouts.csv')

# merge
readout_df_se['Bit']+=100
readout_df = pd.concat([readout_df_gn, readout_df_se])
readout_df.to_csv(r'E:\DNA_analysis\readout_to_bit.csv')
readout_df

,Bit,ReadoutName
0,1,NDB_784
1,2,NDB_755
2,3,NDB_826
3,4,NDB_713
4,5,NDB_865
...,...,...
90,191,NDB_592
91,192,NDB_368
92,193,NDB_436
93,194,NDB_629


In [7]:
from tqdm import tqdm
from ImageAnalysis3.classes import new_decoder
from ImageAnalysis3.classes.new_decoder import batch_process_SpotDecoder

overwrite_decoder = True
verbose_decoder = True
return_decoder = False

pair_search_radius = 300
tuple_search_radius = 300
pair_search_eps = 0.25

## 3.3 prepare kwargs to decode CTP11

In [8]:
%%time

ctp11_data_type = 'combo'
ctp11_lib_name = 'CTP11'
ctp11_decode_args = []
for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):
    print(f"- Preparing decoding args for fov: {_fov_id},", end=' ')
    # load candidate spots for the fov
    cand_spot_filename = os.path.join(cand_spot_folder, ctp11_data_type,
                                      os.path.basename(_save_filename).replace('.hdf5', '_CandSpots.csv'))
    if os.path.isfile(cand_spot_filename):
        _fov_spots_df = pd.read_csv(cand_spot_filename)
    else:
        continue # skip if fov doesn't exist
        
    for _cell_id in np.unique(_fov_spots_df['cell_id']):
        # get decoder filename
        _decoder_filename = os.path.join(decoder_folder, f"Decoded_fov-{_fov_id}_cell-{_cell_id}.hdf5")
        # check if overwrite:
        if not overwrite_decoder and os.path.exists(_decoder_filename):
            # skip if exists
            with h5py.File(_decoder_filename, 'r') as _f:
                if ctp11_lib_name in _f.keys():
                    continue
        # get cell_df
        _cell_spots_df =_fov_spots_df[_fov_spots_df['cell_id']==_cell_id]
        # prepare args
        ctp11_decode_args.append(
            (ctp11_lib_name, _cell_spots_df, ctp11_codebook, readout_df, _decoder_filename,
             pair_search_radius, tuple_search_radius, pair_search_eps, overwrite_decoder, verbose_decoder, return_decoder)
        )
    print(len(ctp11_decode_args))

- Preparing decoding args for fov: 1, 46
- Preparing decoding args for fov: 2, 93
- Preparing decoding args for fov: 3, 158
- Preparing decoding args for fov: 4, 217
- Preparing decoding args for fov: 5, 272
- Preparing decoding args for fov: 6, 337
- Preparing decoding args for fov: 7, 410
- Preparing decoding args for fov: 8, 482
- Preparing decoding args for fov: 9, 550
- Preparing decoding args for fov: 10, 629
- Preparing decoding args for fov: 11, 708
- Preparing decoding args for fov: 12, 788
- Preparing decoding args for fov: 13, 856
- Preparing decoding args for fov: 14, 917
- Preparing decoding args for fov: 15, 972
- Preparing decoding args for fov: 16, 1037
- Preparing decoding args for fov: 17, 1091
- Preparing decoding args for fov: 18, 1131
- Preparing decoding args for fov: 19, 1178
- Preparing decoding args for fov: 20, 1249
- Preparing decoding args for fov: 21, 1323
- Preparing decoding args for fov: 22, 1367
- Preparing decoding args for fov: 23, 1426
- Preparing de

## Test decoding for one cell

In [9]:
%matplotlib inline
from ImageAnalysis3.classes.new_decoder import batch_process_SpotMapper, batch_process_SpotDecoder
sel_cell = 50

# conbinatorial
batch_process_SpotDecoder(*ctp11_decode_args[sel_cell])

- Matching 194 bits to (988, 103) codebook


C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\classes\new_decoder.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.bit_codebook[_matched_bit[0]] = self.codebook[_col].copy()


-- 99 bits matched
- Process (988, 102) codebook into valid pairs
-- 2964 valid pairs detected.
- Searching for spot-pairs within 300nm.
-- find candidate pairs by KDTree
-- filter candidate pairs by codebook
-- 8354 pairs selected.
- Select spot_groups from candidate pairs with radius:300, max_usage=1
-- calculate scores for candSpotPairs


100%|███████████████████████████████████████████████████████████████████████████| 8354/8354 [00:01<00:00, 7466.35it/s]


-- 907 full tuples selected


100%|█████████████████████████████████████████████████████████████████████████| 8354/8354 [00:00<00:00, 227276.48it/s]


-- 546 pairs selected
-- in total 1453 spot_groups detected
- Save decoder of CTP11 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder\Decoded_fov-2_cell-16.hdf5
-- existing info: [] and []
-- save bits
-- save search_radius_pair to attrs
-- save search_radius_tuple to attrs
-- save search_eps to attrs
-- save readoutDf
-- save codebook
-- save candSpots
-- save spotGroups
- Save spot_stats iamage to file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder\Decoded_fov-2_cell-16_CTP11_stats.png


C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\classes\new_decoder.py:401: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['uid', 'homolog', 'channel_0', 'channel_1', 'channel_2', 'region_name',
       'start', 'end', 'chr'],
      dtype='object')]

  _spotGroupDf.to_hdf(self.saveFile, f'{self.codebook_name}/spotGroups', complevel=_complevel, complib=_complib)


# Process All

In [10]:
%%time
from ImageAnalysis3.classes.new_decoder import batch_process_SpotDecoder
import multiprocessing as mp
print(len(ctp11_decode_args))
with mp.Pool(16) as decode_pool:
    decode_results = decode_pool.starmap(batch_process_SpotDecoder, ctp11_decode_args, chunksize=1)
    decode_pool.close()
    decode_pool.join()
    decode_pool.terminate()

13946
CPU times: total: 2min 27s
Wall time: 1h 41min 15s


In [11]:
# Delete args to release RAM
del(ctp11_decode_args)
del(decode_results)

# CTP13 decode

In [12]:
%%time
ctp13_data_type = 'combo'
ctp13_lib_name = 'CTP13'
ctp13_decode_args = []
for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):
    print(f"- Preparing decoding args for fov: {_fov_id},", end=' ')
    # load candidate spots for the fov
    cand_spot_filename = os.path.join(cand_spot_folder, ctp13_data_type,
                                      os.path.basename(_save_filename).replace('.hdf5', '_CandSpots.csv'))
    if os.path.isfile(cand_spot_filename):
        _fov_spots_df = pd.read_csv(cand_spot_filename)
    else:
        continue # skip if fov doesn't exist
        
    for _cell_id in np.unique(_fov_spots_df['cell_id']):
        # get decoder filename
        _decoder_filename = os.path.join(decoder_folder, f"Decoded_fov-{_fov_id}_cell-{_cell_id}.hdf5")
        # check if overwrite:
        if not overwrite_decoder and os.path.exists(_decoder_filename):
            # skip if exists
            with h5py.File(_decoder_filename, 'r') as _f:
                if ctp13_lib_name in _f.keys():
                    continue
        # get cell_df
        _cell_spots_df =_fov_spots_df[_fov_spots_df['cell_id']==_cell_id]
        # prepare args
        ctp13_decode_args.append(
            (ctp13_lib_name, _cell_spots_df, ctp13_codebook, readout_df, _decoder_filename,
             pair_search_radius, tuple_search_radius, pair_search_eps, overwrite_decoder, verbose_decoder, return_decoder)
        )
    print(len(ctp13_decode_args))

- Preparing decoding args for fov: 1, 46
- Preparing decoding args for fov: 2, 93
- Preparing decoding args for fov: 3, 158
- Preparing decoding args for fov: 4, 217
- Preparing decoding args for fov: 5, 272
- Preparing decoding args for fov: 6, 337
- Preparing decoding args for fov: 7, 410
- Preparing decoding args for fov: 8, 482
- Preparing decoding args for fov: 9, 550
- Preparing decoding args for fov: 10, 629
- Preparing decoding args for fov: 11, 708
- Preparing decoding args for fov: 12, 788
- Preparing decoding args for fov: 13, 856
- Preparing decoding args for fov: 14, 917
- Preparing decoding args for fov: 15, 972
- Preparing decoding args for fov: 16, 1037
- Preparing decoding args for fov: 17, 1091
- Preparing decoding args for fov: 18, 1131
- Preparing decoding args for fov: 19, 1178
- Preparing decoding args for fov: 20, 1249
- Preparing decoding args for fov: 21, 1323
- Preparing decoding args for fov: 22, 1367
- Preparing decoding args for fov: 23, 1426
- Preparing de

In [13]:
%matplotlib inline
from ImageAnalysis3.classes.new_decoder import batch_process_SpotMapper, batch_process_SpotDecoder

sel_cell=88

# conbinatorial
batch_process_SpotDecoder(*ctp13_decode_args[sel_cell])

- Matching 194 bits to (966, 99) codebook
-- 95 bits matched
- Process (966, 98) codebook into valid pairs
-- 2898 valid pairs detected.
- Searching for spot-pairs within 300nm.
-- find candidate pairs by KDTree
-- filter candidate pairs by codebook
-- 15253 pairs selected.
- Select spot_groups from candidate pairs with radius:300, max_usage=1
-- calculate scores for candSpotPairs


100%|█████████████████████████████████████████████████████████████████████████| 15253/15253 [00:01<00:00, 7715.64it/s]


-- 1267 full tuples selected


100%|███████████████████████████████████████████████████████████████████████| 15253/15253 [00:00<00:00, 200643.30it/s]


-- 1443 pairs selected
-- in total 2710 spot_groups detected
- Save decoder of CTP13 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder\Decoded_fov-2_cell-82.hdf5
-- existing info: [] and []
-- save bits
-- save search_radius_pair to attrs
-- save search_radius_tuple to attrs
-- save search_eps to attrs
-- save readoutDf
-- save codebook
-- save candSpots
-- save spotGroups


C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\classes\new_decoder.py:401: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['uid', 'homolog', 'channel_0', 'channel_1', 'channel_2', 'region_name',
       'start', 'end', 'chr'],
      dtype='object')]

  _spotGroupDf.to_hdf(self.saveFile, f'{self.codebook_name}/spotGroups', complevel=_complevel, complib=_complib)


- Save spot_stats iamage to file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder\Decoded_fov-2_cell-82_CTP13_stats.png


In [14]:
%%time
from ImageAnalysis3.classes.new_decoder import batch_process_SpotDecoder
import multiprocessing as mp
print(len(ctp13_decode_args))
with mp.Pool(16) as decode_pool:
    decode_results = decode_pool.starmap(batch_process_SpotDecoder, ctp13_decode_args, chunksize=1)
    decode_pool.close()
    decode_pool.join()
    decode_pool.terminate()

13946
CPU times: total: 2min 47s
Wall time: 1h 28min 11s


In [15]:
# Delete args to release RAM
del(ctp13_decode_args)
del(decode_results)

# CTP 12

In [16]:
ctp12_codebook = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\Summary_tables\CTP12-TSS_sequential-codebook.csv', header=0)
ctp12_codebook

,name,id,chr,Stv_91,Stv_32,Stv_92,Stv_33,Stv_94,Stv_35,Stv_95,...,Stv_106,Stv_45,Stv_107,Stv_46,Stv_109,Stv_48,Stv_118,Stv_50,Stv_119,Stv_53
0,5:31098237-31118237,Slc30a3,5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7:45153949-45173949,Slc17a7,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2:158600767-158620767,Slc32a1,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2:70543072-70563072,Gad1,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5:30451932-30471932,Otof,5,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4:124976430-124996430,Rspo1,4,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,15:78196400-78216400,Pvalb,15,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,16:23880958-23900958,Sst,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,10:5629218-5649218,Vip,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,14:34364789-34384789,Sncg,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
ctp12_readout_df = pd.read_csv(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\Summary_tables\CTP12-TSS_unique_readouts.csv', header=0)
ctp12_readout_df

,Bit,ReadoutName
0,1,Stv_91
1,2,Stv_32
2,3,Stv_92
3,4,Stv_33
4,5,Stv_94
5,6,Stv_35
6,7,Stv_95
7,8,Stv_36
8,9,Stv_99
9,10,Stv_37


In [23]:
%%time
from ImageAnalysis3.classes import new_decoder
from ImageAnalysis3.classes.new_decoder import batch_process_SpotMapper


ctp12_lib_name = 'CTP12tss'
ctp12_data_type = 'unique'
ctp12_decode_args = []

for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):
    print(f"- Preparing decoding args for fov: {_fov_id},", end=' ')
    # load candidate spots for the fov
    cand_spot_filename = os.path.join(cand_spot_folder, ctp12_data_type,
                                      os.path.basename(_save_filename).replace('.hdf5','_CandSpots.csv'))
    if os.path.isfile(cand_spot_filename):
        _fov_spots_df = pd.read_csv(cand_spot_filename)
    else:
        print(f"-- skip")
        continue # skip if fov doesn't exist
        
    for _cell_id in np.unique(_fov_spots_df['cell_id']):
        # get decoder filename
        _decoder_filename = os.path.join(decoder_folder, f"Decoded_fov-{_fov_id}_cell-{_cell_id}.hdf5")
        # check if overwrite:
        if not overwrite_decoder and os.path.exists(_decoder_filename):
            # skip if exists
            with h5py.File(_decoder_filename, 'r') as _f:
                if ctp12_lib_name in _f.keys():
                    continue

        # get decoder filename
        _decoder_filename = os.path.join(decoder_folder, f"Decoded_fov-{_fov_id}_cell-{_cell_id}.hdf5")
        # get cell_df
        _cell_spots_df =_fov_spots_df[_fov_spots_df['cell_id']==_cell_id]
        # prepare args
        ctp12_decode_args.append(
            (ctp12_lib_name, _cell_spots_df, ctp12_codebook, ctp12_readout_df, _decoder_filename,
             pair_search_radius, pair_search_eps, overwrite_decoder, verbose_decoder, return_decoder)
        )
    print(len(ctp12_decode_args))
    

- Preparing decoding args for fov: 1, 46
- Preparing decoding args for fov: 2, 93
- Preparing decoding args for fov: 3, 158
- Preparing decoding args for fov: 4, 217
- Preparing decoding args for fov: 5, 272
- Preparing decoding args for fov: 6, 337
- Preparing decoding args for fov: 7, 410
- Preparing decoding args for fov: 8, 482
- Preparing decoding args for fov: 9, 550
- Preparing decoding args for fov: 10, 629
- Preparing decoding args for fov: 11, 708
- Preparing decoding args for fov: 12, 786
- Preparing decoding args for fov: 13, 854
- Preparing decoding args for fov: 14, 915
- Preparing decoding args for fov: 15, 970
- Preparing decoding args for fov: 16, 1034
- Preparing decoding args for fov: 17, 1088
- Preparing decoding args for fov: 18, 1128
- Preparing decoding args for fov: 19, 1175
- Preparing decoding args for fov: 20, 1245
- Preparing decoding args for fov: 21, 1318
- Preparing decoding args for fov: 22, 1361
- Preparing decoding args for fov: 23, 1420
- Preparing de

- Preparing decoding args for fov: 192, 10988
- Preparing decoding args for fov: 193, 11061
- Preparing decoding args for fov: 194, 11138
- Preparing decoding args for fov: 195, 11197
- Preparing decoding args for fov: 196, 11249
- Preparing decoding args for fov: 197, 11315
- Preparing decoding args for fov: 198, 11388
- Preparing decoding args for fov: 199, 11441
- Preparing decoding args for fov: 200, 11506
- Preparing decoding args for fov: 201, 11579
- Preparing decoding args for fov: 202, 11651
- Preparing decoding args for fov: 203, 11698
- Preparing decoding args for fov: 204, 11760
- Preparing decoding args for fov: 205, 11803
- Preparing decoding args for fov: 206, 11855
- Preparing decoding args for fov: 207, 11908
- Preparing decoding args for fov: 208, 11967
- Preparing decoding args for fov: 209, 12028
- Preparing decoding args for fov: 210, 12080
- Preparing decoding args for fov: 211, 12139
- Preparing decoding args for fov: 212, 12188
- Preparing decoding args for fov:

In [24]:
%%time
print(len(ctp12_decode_args))
with mp.Pool(20) as decode_pool:
    decode_results = decode_pool.starmap(batch_process_SpotMapper, ctp12_decode_args, chunksize=1)
    decode_pool.close()
    decode_pool.join()
    decode_pool.terminate()

14002
CPU times: total: 4.42 s
Wall time: 2min 49s
